# 쿠텐베르크 동화를 활용한 영단어 Embedding 

### 데이터 불러오기 및 정규화

In [1]:
import requests
import re

In [2]:
res = requests.get('https://www.gutenberg.org/files/2591/2591-0.txt')

In [3]:
grimm = res.text[2801:530661]

In [4]:
grimm = re.sub(r'[^a-zA-Z\. ]', ' ', grimm)

In [5]:
sentences = grimm.split(". ")              # 문장 단위로 자르기

In [6]:
data = [s.split() for s in sentences]     # 단어 단위로 자르기

In [7]:
data[0]

['SECOND',
 'STORY',
 'THE',
 'SALAD',
 'THE',
 'STORY',
 'OF',
 'THE',
 'YOUTH',
 'WHO',
 'WENT',
 'FORTH',
 'TO',
 'LEARN',
 'WHAT',
 'FEAR',
 'WAS',
 'KING',
 'GRISLY',
 'BEARD',
 'IRON',
 'HANS',
 'CAT',
 'SKIN',
 'SNOW',
 'WHITE',
 'AND',
 'ROSE',
 'RED',
 'THE',
 'BROTHERS',
 'GRIMM',
 'FAIRY',
 'TALES',
 'THE',
 'GOLDEN',
 'BIRD',
 'A',
 'certain',
 'king',
 'had',
 'a',
 'beautiful',
 'garden',
 'and',
 'in',
 'the',
 'garden',
 'stood',
 'a',
 'tree',
 'which',
 'bore',
 'golden',
 'apples']

### Word2Vec 훈련시키기

In [8]:
from gensim.models.word2vec import Word2Vec

In [9]:
model = Word2Vec(data,               # 리스트 형태의 데이터
                 sg=1,               # 0: CBOW, 1: skip-gram
                 vector_size=100,    # 벡터 크기 하나의 단어를 몇 차원 벡터로 표현할 것인가
                 window=3,           # 고려할 앞뒤 범위(앞 뒤 3 단어)
                 min_count=3,        # 사용할 단어의 최소 빈도(3회 이하 단어 무시)
                 workers=4           # 동시에 처리할 작업 수(코어 수와 비슷하게 설정)
                )

#### 모델 저장 및 로드

In [10]:
model.save('word2vec.model')

In [11]:
model = Word2Vec.load('word2vec.model')

#### wv 메소드로 유사어 확인

In [12]:
model.wv["princess"]

array([-0.13470188,  0.11796378, -0.06052293,  0.13504401,  0.01958002,
       -0.31074673,  0.11877873,  0.5310646 , -0.21852227, -0.2580635 ,
        0.0372058 , -0.32491234,  0.02511626,  0.109215  ,  0.14125998,
       -0.21415263,  0.03035061, -0.11118113, -0.1481159 , -0.23580956,
        0.22472575,  0.1551313 ,  0.17465192, -0.18112889,  0.01991536,
       -0.01570466,  0.03458815, -0.05552115, -0.14782451, -0.08897363,
        0.07145429, -0.1360997 ,  0.12224001, -0.16139035, -0.16858649,
        0.1852571 ,  0.03969387, -0.043217  , -0.06977359, -0.11906592,
        0.09833706, -0.09055311, -0.03794726,  0.07245405, -0.06045794,
        0.02491899, -0.1353533 , -0.09347339,  0.18845795,  0.05946556,
        0.0934922 , -0.0616981 , -0.07652557, -0.10545052,  0.14572676,
        0.10431336,  0.1188615 ,  0.020956  , -0.20838934,  0.01984456,
       -0.06827766,  0.07428323,  0.14685601, -0.19489628, -0.2774837 ,
        0.10785498,  0.05206594,  0.37525922, -0.25717565,  0.27

In [13]:
model.wv.similarity("princess", "queen")

0.9688886

In [14]:
model.wv.most_similar('princess')

[('palace', 0.98224276304245),
 ('fox', 0.9820863008499146),
 ('dwarf', 0.9820170402526855),
 ('wedding', 0.9807780981063843),
 ('boy', 0.9805260896682739),
 ('prince', 0.9786450266838074),
 ('huntsmen', 0.9785482287406921),
 ('same', 0.9785082340240479),
 ('fairy', 0.9777272939682007),
 ('dog', 0.9771775007247925)]

In [15]:
model.wv.most_similar(positive=['man', 'princess'], negative=['woman'])
# man + princess - woman prince를 기대값으로 하지만 학습량이 적어서 정확도 떨어짐

[('cat', 0.9616175293922424),
 ('bird', 0.9612706899642944),
 ('eldest', 0.9607287049293518),
 ('prince', 0.9594089388847351),
 ('frog', 0.9572567939758301),
 ('fisherman', 0.9571784734725952),
 ('gardener', 0.9562231302261353),
 ('boy', 0.955575704574585),
 ('bride', 0.9554721713066101),
 ('shepherd', 0.9550653100013733)]

### Keras에서 Embedding하여 결과 비교

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding

In [23]:
model.wv.vectors.shape # 단어 수, 벡터 크기

(2446, 100)

In [17]:
NUM_WORDS, EMB_DIM = model.wv.vectors.shape
emb = Embedding(input_dim=NUM_WORDS, output_dim=EMB_DIM,
               trainable=False, weights=[model.wv.vectors])

In [18]:
net = Sequential()
net.add(emb)
net.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         244600    
                                                                 
Total params: 244,600
Trainable params: 0
Non-trainable params: 244,600
_________________________________________________________________


#### gensim 과 비교

In [19]:
i = model.wv.index_to_key.index("princess")
net.predict([i])

1/1 [==============================] - 0s 166ms/step


array([[-0.13470188,  0.11796378, -0.06052293,  0.13504401,  0.01958002,
        -0.31074673,  0.11877873,  0.5310646 , -0.21852227, -0.2580635 ,
         0.0372058 , -0.32491234,  0.02511626,  0.109215  ,  0.14125998,
        -0.21415263,  0.03035061, -0.11118113, -0.1481159 , -0.23580956,
         0.22472575,  0.1551313 ,  0.17465192, -0.18112889,  0.01991536,
        -0.01570466,  0.03458815, -0.05552115, -0.14782451, -0.08897363,
         0.07145429, -0.1360997 ,  0.12224001, -0.16139035, -0.16858649,
         0.1852571 ,  0.03969387, -0.043217  , -0.06977359, -0.11906592,
         0.09833706, -0.09055311, -0.03794726,  0.07245405, -0.06045794,
         0.02491899, -0.1353533 , -0.09347339,  0.18845795,  0.05946556,
         0.0934922 , -0.0616981 , -0.07652557, -0.10545052,  0.14572676,
         0.10431336,  0.1188615 ,  0.020956  , -0.20838934,  0.01984456,
        -0.06827766,  0.07428323,  0.14685601, -0.19489628, -0.2774837 ,
         0.10785498,  0.05206594,  0.37525922, -0.2

#### 모델 저장 및 로드

In [20]:
from gensim.models import KeyedVectors

In [21]:
model.wv.save_word2vec_format('eng_w2v')

In [22]:
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v")

### 사전학습 Word2Vec 가져오기
---
구글 사전 훈련 word2vec 호출해서 사용하기

모델 다운로드 경로 : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin 파일 경로', binary=True)

wv.similarity()에 두 단어를 넘겨주면 코사인 유사도를 구할 수 있다.
wv.most_similar()에 단어를 넘겨주면 가장 유사한 단어를 추출할 수 있다.
.wv_most_similar() 에 positive와 negative라는 옵션을 넘겨줄 수 있다.